# Categorize raw job titles into pre-defined job functions and job levels
tutorials: https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/Clean%20and%20Standardize%20Data.ipynb

# Load library

In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import json

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# Define Model and output format

In [3]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOpenAI(temperature=0, max_tokens=1000)

In [4]:
# Define reponse structured as prompt template
response_schemas = [
    ResponseSchema(name="input_job titles", description="This is the raw job titles from the user"),
    ResponseSchema(name="job functions", description="This is the job function you feel is most closely matched to the users input"),
    ResponseSchema(name="job level", description="This is the job level you feel is most closely matched to the users input"),
    ResponseSchema(name="match_score",  description="A score 0-100 of how close you think the match is between user input and your match")
]

# parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"input_job titles": string  // This is the raw job titles from the user
	"job functions": string  // This is the job function you feel is most closely matched to the users input
	"job level": string  // This is the job level you feel is most closely matched to the users input
	"match_score": string  // A score 0-100 of how close you think the match is between user input and your match
}
```


In [12]:
template = """
You will be given a list of job titles from a user, delineated by ";"
For each job title, find the best corresponding match on the list of standardized job functions and job levels.
The closest match will be the one with the closest semantic meaning. Not just string similarity.

{format_instructions}

Wrap your final output with cloased and open brackets (a list of json objects). Nothing else. 

job_titles INPUT: {job_titles}

job_functions:{job_functions}
job_levels: {job_levels}

YOUR RESPONSE:

"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["job_titles", "job_functions", "job_levels"],
    partial_variables={"format_instructions": format_instructions}
)

# Define Inputs

In [13]:
# standardized job functions and levels
df = pd.read_csv('job_functions.csv')
job_functions = ", ".join(df['functions'].values)
job_functions

'unknown, finance, engineering, customer success, retail, operations, design, sales, marketing, administration, support, product, general management, HR, commercial management, analytics, recruiting, legal'

In [8]:
# standardized job functions and levels
df = pd.read_csv('job_levels.csv')
job_levels = ", ".join(df['level'].values)
job_levels

'Director, line, CXO, VP, Manager'

In [9]:
# Input

df = pd.read_csv('job_titles_uncoded.csv')
# df = pd.read_csv('job_titles_coded.csv')
job_titles = ";".join(df['job_title'].values[10:19])
job_titles

'administrateur reseaux   developpeur windev;production lead;manager;senior director global dtc;director business integration;head muckety muck;vice president;network administrator;vice president   development'

In [14]:
user_input = job_titles

_input = prompt.format_prompt(job_titles=user_input, job_functions=job_functions, job_levels = job_levels)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain.schema.HumanMessage'>
---------------------------

You will be given a list of job titles from a user, delineated by ";"
For each job title, find the best corresponding match on the list of standardized job functions and job levels.
The closest match will be the one with the closest semantic meaning. Not just string similarity.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"input_job titles": string  // This is the raw job titles from the user
	"job functions": string  // This is the job function you feel is most closely matched to the users input
	"job level": string  // This is the job level you feel is most closely matched to the users input
	"match_score": string  // A score 0-100 of how close you think the match is between user input and your match
}
```

Wrap your final output with cloased and open brackets (a list of json obj

## Run model 


In [15]:
output = chat_model(_input.to_messages())
print (type(output))
print (output.content)

<class 'langchain.schema.AIMessage'>
```json
[
	{
		"input_job titles": "administrateur reseaux",
		"job functions": "engineering",
		"job level": "line",
		"match_score": "80"
	},
	{
		"input_job titles": "developpeur windev",
		"job functions": "engineering",
		"job level": "line",
		"match_score": "90"
	},
	{
		"input_job titles": "production lead",
		"job functions": "operations",
		"job level": "line",
		"match_score": "85"
	},
	{
		"input_job titles": "manager",
		"job functions": "general management",
		"job level": "Manager",
		"match_score": "80"
	},
	{
		"input_job titles": "senior director global dtc",
		"job functions": "general management",
		"job level": "Director",
		"match_score": "90"
	},
	{
		"input_job titles": "director business integration",
		"job functions": "general management",
		"job level": "Director",
		"match_score": "95"
	},
	{
		"input_job titles": "head muckety muck",
		"job functions": "unknown",
		"job level": "CXO",
		"match_score": "50"
	},
	{
		"inp

In [16]:
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
    json_string = json_string.split("```")[0].strip()
else:
    json_string = output.content

In [17]:
print(json_string)

[
	{
		"input_job titles": "administrateur reseaux",
		"job functions": "engineering",
		"job level": "line",
		"match_score": "80"
	},
	{
		"input_job titles": "developpeur windev",
		"job functions": "engineering",
		"job level": "line",
		"match_score": "90"
	},
	{
		"input_job titles": "production lead",
		"job functions": "operations",
		"job level": "line",
		"match_score": "85"
	},
	{
		"input_job titles": "manager",
		"job functions": "general management",
		"job level": "Manager",
		"match_score": "80"
	},
	{
		"input_job titles": "senior director global dtc",
		"job functions": "general management",
		"job level": "Director",
		"match_score": "90"
	},
	{
		"input_job titles": "director business integration",
		"job functions": "general management",
		"job level": "Director",
		"match_score": "95"
	},
	{
		"input_job titles": "head muckety muck",
		"job functions": "unknown",
		"job level": "CXO",
		"match_score": "50"
	},
	{
		"input_job titles": "vice president",
		"job func

In [18]:
# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)
structured_data

[{'input_job titles': 'administrateur reseaux',
  'job functions': 'engineering',
  'job level': 'line',
  'match_score': '80'},
 {'input_job titles': 'developpeur windev',
  'job functions': 'engineering',
  'job level': 'line',
  'match_score': '90'},
 {'input_job titles': 'production lead',
  'job functions': 'operations',
  'job level': 'line',
  'match_score': '85'},
 {'input_job titles': 'manager',
  'job functions': 'general management',
  'job level': 'Manager',
  'match_score': '80'},
 {'input_job titles': 'senior director global dtc',
  'job functions': 'general management',
  'job level': 'Director',
  'match_score': '90'},
 {'input_job titles': 'director business integration',
  'job functions': 'general management',
  'job level': 'Director',
  'match_score': '95'},
 {'input_job titles': 'head muckety muck',
  'job functions': 'unknown',
  'job level': 'CXO',
  'match_score': '50'},
 {'input_job titles': 'vice president',
  'job functions': 'general management',
  'job lev

In [20]:
pd.concat([pd.DataFrame(structured_data), df.loc[10:19, 'functions']], axis=1)

,input_job titles,job functions,job level,match_score,functions
0,administrateur reseaux,engineering,line,80,NaN
1,developpeur windev,engineering,line,90,NaN
2,production lead,operations,line,85,NaN
3,manager,general management,Manager,80,NaN
4,senior director global dtc,general management,Director,90,NaN
5,director business integration,general management,Director,95,NaN
6,head muckety muck,unknown,CXO,50,NaN
7,vice president,general management,VP,80,NaN
8,network administrator,engineering,line,85,NaN
9,vice president development,engineering,VP,90,NaN
